In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymc3 as pm
import arviz as az
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [ ]:
#Reading the data.csv file and filtering it
df = pd.read_csv(r"C:\Users\Farooqi\Downloads\data.csv")
df = df[df['Company'] == 'META']
df.drop(columns=['Date','Company'], inplace=True)

In [ ]:
# Defining the model
with pm.Model() as gamma_model:
    a = pm.Normal("a", 0, 0.5)
    beta_volume = pm.Normal("beta_volume", mu=5, sigma=1)
    beta_open = pm.Normal("beta_open", mu=5, sigma=1)
    beta_high = pm.Normal("beta_high", mu=5, sigma=1)
    beta_low = pm.Normal("beta_low", mu=5, sigma=1)
    intercept = pm.Normal('intercept', mu=5, sigma=1)
    shape = pm.Uniform("shape", 0, 50)
    
    # Log Link Function
    μ = pm.math.exp(a + beta_volume * df['Volume'] + beta_open * df['Open'] + beta_high * df['High'] + beta_low * df['Low'] + intercept)

    likelihood = pm.Gamma("Close", alpha=shape, beta=pm.math.exp(a + intercept), observed=df['Close'])

    map_estimate = pm.find_MAP(progressbar=False)
    trace = pm.sample(1000, chains=1, cores=4, return_inferencedata=False)


In [ ]:
print(list(trace))

In [ ]:
#Getting the value for PPC using trace
with gamma_model:
    ppc = pm.sample_posterior_predictive(
        trace, random_seed=42, progressbar=True
    )

In [ ]:
with gamma_model: 
    trace_updated = az.from_pymc3(trace, posterior_predictive=ppc)

In [ ]:
print(trace_updated)

In [ ]:
print(trace_updated["posterior"])
print(trace_updated['posterior_predictive'])
print(trace_updated['log_likelihood'])
print(trace_updated['sample_stats'])
print(trace_updated['observed_data'])


In [ ]:
#Defining Trace plots
az.style.use("arviz-darkgrid")
with gamma_model:
  az.plot_trace(trace_updated, figsize=(17,10), legend=True)

In [ ]:
#Definig Density plots
az.style.use("arviz-darkgrid")
with gamma_model:
  az.plot_posterior(trace_updated,textsize=16)

In [ ]:
az.style.use("arviz-darkgrid")
with gamma_model:
  az.plot_ppc(trace_updated)

In [ ]:
ppc = pm.sample_posterior_predictive(trace, samples=1000, model=gamma_model)
# Extracting mean of predicted values
predicted_values = np.mean(ppc['Close'], axis=0)

# Calculating RMSE
rmse = np.sqrt(np.mean((df['Close'] - predicted_values)**2))
print("RMSE:", rmse)

# Calculating Mean Absolute Error (MAE)
mae = np.mean(np.abs(df['Close'] - predicted_values))
print("Mean Absolute Error (MAE):", mae)